In [26]:
import pandas as pd
import numpy as np 
import configparser
import re
import uuid

In [27]:
# Get data from .csv files

config = configparser.ConfigParser()
config.read('../config.ini')

books_csv_url = config['URLIntermediate']['books']
users_csv_url = config['URLIntermediate']['users']
ratings_csv_url = config['URLIntermediate']['ratings']

df_books = pd.read_csv(books_csv_url)
df_users = pd.read_csv(users_csv_url)
df_rating = pd.read_csv(ratings_csv_url)

fact_table_url = config['URLProcessed']['fact_table']
dim_table_user_url = config['URLProcessed']['dim_table_user']
dim_table_location_url = config['URLProcessed']['dim_table_location']
dim_table_book_url = config['URLProcessed']['dim_table_book']


In [34]:
test = df_rating[df_rating['User-ID'] == 8]
test

,User-ID,ISBN,Book-Rating
8700,8,2005018,5
8701,8,60973129,0
8702,8,374157065,0
8703,8,393045218,0
8704,8,399135782,0
8705,8,425176428,0
8706,8,671870432,0
8707,8,679425608,0
8708,8,771025661,0
8709,8,771074670,0


In [29]:
# Create final tables 

df_fact_table = pd.DataFrame({
                    'rating_id': pd.Series(dtype='str'),
                   'user_id': pd.Series(dtype='str'),
                   'location_id': pd.Series(dtype='str'),
                   'book_id': pd.Series(dtype='str'),
                   'rating': pd.Series(dtype='double'),
                   })

df_dim_table_user = pd.DataFrame({
                    'user_id': pd.Series(dtype='str'),
                   'age': pd.Series(dtype='int')
                   })

df_dim_table_location = pd.DataFrame({
                    'location_id': pd.Series(dtype='str'),
                   'city': pd.Series(dtype='str'),
                   'department': pd.Series(dtype='str'),
                   'country': pd.Series(dtype='str')
                   })

df_dim_table_book = pd.DataFrame({
                   'isbn': pd.Series(dtype='str'),
                   'title': pd.Series(dtype='str'),
                   'author': pd.Series(dtype='str'),
                   'year': pd.Series(dtype='int'),
                   'publisher' : pd.Series(dtype='str')
                   })

In [35]:
# Dimensional model set - up 
# 1. Join all data 


# df_source = (
#     df_rating.merge(df_books, left_on='ISBN', right_on='ISBN', how='left')
#        .merge(df_users, left_on='User-ID', right_on='User-ID', how='left')
#     #    [['id', 'id_employee', 'id_function', 'id_department', 'id_education_level']]
# )

df_source = (
    df_users.merge(df_rating, left_on='User-ID', right_on='User-ID', how='inner')
       .merge(df_books, left_on='ISBN', right_on='ISBN', how='inner')
    #    [['id', 'id_employee', 'id_function', 'id_department', 'id_education_level']]
)


df_source['User-ID'] = df_source['User-ID'].astype('str')
df_source['ISBN'] = df_source['ISBN'].astype('str')
df_source.head(5)
df_source.dtypes

User-ID                object
Age                     int64
location_city          object
location_department    object
location_country       object
ISBN                   object
Book-Rating             int64
Book-Title             object
Book-Author            object
Year-Of-Publication     int64
Publisher              object
dtype: object

In [36]:
test = df_source[df_source['ISBN'] == '1552041778']
test

,User-ID,Age,location_city,location_department,location_country,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
80,8,0,timmins,ontario,canada,1552041778,5,Jane Doe,R. J. Kaiser,1999,Mira Books


In [32]:
# 2. Set rating_id

df_source = df_source.drop_duplicates().reset_index(drop=True)
df_source['rating_id'] = df_source.apply(lambda row: str(uuid.uuid4()), axis=1)


In [33]:
# 3. Dim user table

df_user_temp = df_source[['User-ID','Age']].drop_duplicates().reset_index(drop=True)

df_dim_table_user['user_id'] = df_user_temp['User-ID'].astype('str')
df_dim_table_user['age'] = df_user_temp['Age']


KeyError: "['Age'] not in index"

In [ ]:

# 3. Dim location table

df_location_temp = df_source[['location_city','location_department','location_country']].drop_duplicates().reset_index(drop=True)

df_dim_table_location['city'] = df_location_temp['location_city']
df_dim_table_location['department'] = df_location_temp['location_department']
df_dim_table_location['country'] = df_location_temp['location_country']

df_dim_table_location['location_id'] = df_dim_table_location.apply(lambda row: str(uuid.uuid4()), axis=1)

df_dim_table_location

,location_id,city,department,country,location_city,location_department,location_country
0,900797bf-7ed2-453f-a569-3dd5c23f94cf,seattle,washington,usa,seattle,washington,usa
1,eb79254f-5169-47c6-8df5-107c92aa8509,h,new south wales,australia,h,new south wales,australia
2,e3f8bed7-3549-435d-b668-45d83a42332f,rijeka,n/a,croatia,rijeka,n/a,croatia
3,c1cd0d31-937e-45d9-aaff-21fa14f256a3,paris,n/a,france,paris,n/a,france
4,5eeab34d-809e-4b7b-8b27-56a6e57742eb,salzburg,salzburg,austria,salzburg,salzburg,austria
...,...,...,...,...,...,...,...
24840,fed92f09-5ace-4513-8e4d-c93bc34cd75f,newport,gwent,united kingdom,newport,gwent,united kingdom
24841,b304cb30-1d01-41f9-b5f3-989cc9f169c9,beverly hills,yukon territory,vietnam,beverly hills,yukon territory,vietnam
24842,76c2a250-5e32-4959-88b4-8aaef8d21954,reeves,louisiana,usa,reeves,louisiana,usa
24843,929a8b8e-fbb6-4201-84dc-947ab2900114,wakeman,ohio,usa,wakeman,ohio,usa


In [ ]:

# 3. Dim book table

df_book_temp = df_source[['ISBN','Book-Title','Book-Author', 'Year-Of-Publication', 'Publisher']].drop_duplicates().reset_index(drop=True)

df_dim_table_book['isbn'] = df_book_temp['ISBN']
df_dim_table_book['title'] = df_book_temp['Book-Title']
df_dim_table_book['author'] = df_book_temp['Book-Author']
df_dim_table_book['year'] = df_book_temp['Year-Of-Publication']
df_dim_table_book['publisher'] = df_book_temp['Publisher']

In [ ]:
df_fact_table = (
            df_source.merge(df_dim_table_book, left_on='ISBN', right_on= 'isbn') 
             .merge(df_dim_table_location, left_on=['location_city','location_department','location_country'],
                        right_on=['city','department','country'])
             .merge(df_dim_table_user, left_on='User-ID', right_on= 'user_id') 

             [['rating_id', 'user_id', 'location_id', 'isbn']]
)

df_fact_table

,rating_id,user_id,location_id,isbn
0,2a3c410d-4cb8-4e5f-bb58-c5ca83b04dce,276726,900797bf-7ed2-453f-a569-3dd5c23f94cf,155061224
1,a526b731-0b6b-47df-88cd-c53eb8dadc47,159181,900797bf-7ed2-453f-a569-3dd5c23f94cf,155061224
2,cd832eb1-2c7f-4b1d-8bc8-05a0e271775b,269566,900797bf-7ed2-453f-a569-3dd5c23f94cf,553561618
3,43b47e97-af3d-4c23-ac91-fd21258d4c28,269566,900797bf-7ed2-453f-a569-3dd5c23f94cf,451192001
4,964d771c-9d95-4665-9e26-62bc8a3c0c15,269566,900797bf-7ed2-453f-a569-3dd5c23f94cf,61054151
...,...,...,...,...
1044637,6fbf5d05-b9c6-4d52-bc14-2deca503bb3f,276207,6bd0bbb7-895b-42f9-9bf6-b385b9b58135,9072186206
1044638,4475686b-fa9a-4093-88c6-f5be3660b36b,276216,638d072b-4d35-4d3b-b3bb-f0247ff8a69c,9700509222
1044639,b6101402-8d87-4baa-86a4-9ab9d9bdb0c2,276292,335fdd62-e5bc-44ee-a45a-8e621ff88143,3442234379
1044640,c3bb53a6-f4e8-456e-b0d9-ebdc5f1d4668,276448,d42bdacc-453a-4b81-a9e9-8a8630609d7c,8886661975


In [ ]:

df_dim_table_book

,isbn,title,author,year,publisher
0,155061224,Rites of Passage,Judith Rae,2001.0,Heinle
1,446520802,The Notebook,Nicholas Sparks,1996.0,Warner Books
2,521795028,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001.0,Cambridge University Press
3,2080674722,Les Particules Elementaires,Michel Houellebecq,1998.0,Flammarion
4,3257224281,NaN,NaN,NaN,NaN
...,...,...,...,...,...
304431,517145553,Mostly Harmless,Douglas Adams,1995.0,Random House Value Pub
304432,1575660792,Gray Matter,Shirley Kennett,1996.0,Kensington Publishing Corporation
304433,590907301,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997.0,Apple
304434,679752714,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997.0,Vintage Books USA
